# Comparison based on variability

## Load required libraries

In [1]:
# To reload external files automatically (ex: utils)    
from pathlib import Path     
import numpy as np    
import pandas as pd     
import netCDF4 as nc  
from netCDF4 import Dataset   
import datetime as dt
import calendar as cld
import matplotlib.pyplot as plt
import xarray as xr
import proplot as plot  # New plot library (https://proplot.readthedocs.io/en/latest/)
from scipy import stats
import seaborn as sns
from seaborn import scatterplot
import matplotlib.cm as cm
import itertools
import os

## Set variables

In [14]:
"""
We define the threshold used to deduce number of days with snow over land from snow cover fraction
threshold = '0' <=> 0%, threshold = '1' <=> 1%, threshold = '3' <=> 3% 
"""
threshold = '0' 

# Set elevation ranges list
bounds = ['0m_5000m','1000m_5000m','1000m_1500m', '1500m_2000m', '2000m_5000m']  

# Set input data path
path_cmip_raw_data = '/home/hchaabani/Data/Snow/snc/cmip6/raw_data'
path_input_cmip6 = '/home/hchaabani/Data/Snow/snc/cmip6/modified_data/monthly/spatial_averages_by_alt_bounds_monthly_values/threshold_'+threshold
path_input_era5 = '/home/hchaabani/Data/Snow/snc/Reanalysis/ERA5/modified_data/monthly/spatial_averages_by_alt_bounds_monthly_values/threshold_'+threshold
path_input_era5_land = '/home/hchaabani/Data/Snow/snc/Reanalysis/ERA-LAND/modified_data/monthly/spatial_averages_by_alt_bounds_monthly_values/threshold_'+threshold
path_input_sim_35km = '/home/hchaabani/Data/Snow/snc/SIM_35km/Modified_data/monthly/spatial_averages_by_alt_bounds_monthly_values/threshold_'+threshold
path_input_esa_gf = '/home/hchaabani/Data/Snow/snc/Satellite/ESA_CCI/modified_data/monthly/spatial_averages_by_alt_bounds_monthly_values/threshold_'+threshold

                     

# Set path of directories in which results will be stored
output_path = '/home/hchaabani/Hamid/PHD-mountain-climate/First_Part/Results/data/variabilities trends/coeff variability/spatial_avergaes/threshold_'+threshold

# Set others useful variables                 
period = slice('1982-01-01', '2014-12-31')  
cmip6_labels = os.listdir(path_cmip_raw_data)
products_labels = ['esa_gf','era5_land','era5', 
                   'SIM_35km']+cmip6_labels

paths_inputs = {'esa_gf':path_input_esa_gf,'era5_land':path_input_era5_land,'era5':path_input_era5, 
                'SIM_35km':path_input_sim_35km} | {prod:path_input_cmip6 for prod in cmip6_labels}

dic_data = {bound:{key:'data' for key in products_labels} for bound in bounds}   
months_labels = {1:'JAN',2:'FEB',3:'MAR',4:'APR',5:'MAY',6:'JUN',7:'JUL',8:'AUG',
                 9:'SEP',10:'OCT',11:'NOV', 12:'DEC', }

parameters = ['snc_monthly_value_15','ext_days_with_snow_15']
bounds_titles = ['alt >= 0m', 'alt > 1000m', '1000m-1500m', '1500m-2000m', 'alt > 2000m']

imonths = [1,2,3,4,5,6,7,8,9,10,11,12]

## Importing and preparing data

In [23]:
for bound in bounds:
    for prod in products_labels:    
        dic_data[bound][prod] = xr.open_dataset(paths_inputs[prod]+ '/where_alt_between_'+ bound + '/'+'snc_monthly_'+prod+'_'+bound+'.nc').sel(time=period).load()    

<div class="alert alert-block alert-success"; background-color:red> There are some missing dates in the time dimension of some products, we will add these dates and assign them nan values in order to homogenize the timesteps in all products </div> 

In [26]:
"""
We detect missing dates by applying the difference function between a complete dataset and the dataset that is not
complete      
""" 
for bound in bounds:
    for key in products_labels: 
        missing_dates = set(dic_data[bound]['era5_land'].time.values).difference(set(dic_data[bound][key].time.values))
        missing_dates = list(missing_dates)   
    
        if len(missing_dates) !=0:
            # Then we convert the list of missing date to warray dataarray 
            missing_date_ds = xr.DataArray(missing_dates, dims=["time"], coords=[missing_dates])
        
            # We constract a dataaray of the whole dates period  
            full_dates = xr.concat([dic_data[bound][key].time, missing_date_ds], dim="time")
    
            # We reindex the original dataset and fill empty values with nan
            dic_data[bound][key] = dic_data[bound][key].reindex(time=full_dates, fill_value=np.nan).sortby("time")       

<div class="alert alert-block alert-success"; background-color:red> We keep just the dates where the information about spatial meanis available for all products, if in one date a product gives no information (nan), we should assign nan value to all other products in this specific timestemp. </div> 

In [27]:
for bound in bounds:
    dic_data[bound]['esa_gf'] = dic_data[bound]['esa_gf'].drop_duplicates(dim='time',keep='first')

In [28]:
"""
We keep only the monthly values which are available for all 
product, in order to compute the metrics over the sames periods
""" 
for bound in bounds:
    for par in parameters: 
        for date in dic_data[bound]['era5_land'].time.values:
            for prod in products_labels:
                if np.isnan(dic_data[bound][prod].loc[{'time':date}][par].values):
                    for prod in products_labels:  
                        dic_data[bound][prod][par].loc[{'time':date}] = np.nan 
    print('done for the bound: '+bound)

done for the bound: 0m_5000m
done for the bound: 1000m_5000m
done for the bound: 1000m_1500m
done for the bound: 1500m_2000m
done for the bound: 2000m_5000m


##  Variation coefficient computing

In [29]:
cv_dic = {par:{mon:'data' for mon in months_labels.values()} for par in parameters}

In [30]:
for par in parameters:
    for mon in imonths:
        df = pd.DataFrame(columns=['bound']+products_labels)
        df['bound'] = bounds
        for prod in products_labels:
            L=[]            
            for bound in bounds:       
                mean = np.mean(dic_data[bound][prod][par].groupby('time.month')[mon].dropna(dim='time').values)
                std = np.std(dic_data[bound][prod][par].groupby('time.month')[mon].dropna(dim='time').values)          
                cv = std/mean * 100
                L.append(np.round(cv,2))
            
            df[prod] = L                 
                
        cv_dic[par][months_labels[mon]] = df 
        
        print('done for: '+par+'/'+str(mon))

done for: snc_monthly_value_15/1
done for: snc_monthly_value_15/2
done for: snc_monthly_value_15/3
done for: snc_monthly_value_15/4
done for: snc_monthly_value_15/5
done for: snc_monthly_value_15/6
done for: snc_monthly_value_15/7
done for: snc_monthly_value_15/8


/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: R

done for: snc_monthly_value_15/9
done for: snc_monthly_value_15/10
done for: snc_monthly_value_15/11
done for: snc_monthly_value_15/12
done for: ext_days_with_snow_15/1
done for: ext_days_with_snow_15/2
done for: ext_days_with_snow_15/3
done for: ext_days_with_snow_15/4
done for: ext_days_with_snow_15/5
done for: ext_days_with_snow_15/6


/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: R

done for: ext_days_with_snow_15/7
done for: ext_days_with_snow_15/8
done for: ext_days_with_snow_15/9
done for: ext_days_with_snow_15/10


/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100
/tmp/ipykernel_28550/2416423525.py:10: RuntimeWarning: invalid value encountered in scalar divide
  cv = std/mean * 100


done for: ext_days_with_snow_15/11
done for: ext_days_with_snow_15/12


## Variation coefficient storing

In [ ]:
for par in parameters:          
    for mon in imonths:     
        cv_dic[par][months_labels[mon]].to_excel(output_path+'/variability_coeff_'+par+'_'+months_labels[mon]+'.xlsx')